DST: Are you able to reproduce this issue? The user is noting participants who have "Skip"  to the parent question ""Have you smoked at least 100 cigarettes in your entire life? (There are 20 cigarettes in a pack.)" (concept ID 1585857) still have response options for the branching logic - "Do you now smoke cigarettes every day, some days, or not at all?" (concept ID 1585860).

I was exploring the survey response structure and had a few questions.

When looking at the first cigarette question on the "Lifestyle" survey. People who answered "yes" to smoking 100 cigs can also have an NA value for the follow-up question on "Smoke Frequency (Do you now smoke cigarettes every day, some days, or not at all?)"? Why are these people not labeled as "PMI: Skip"?

When looking at everyone who didn't answer "yes" to smoking 100 cigs (No, Dont Know, Prefer Not To Answer, and Skip). Why do people who have "PMI: Skip" responses to 100 Cigs Lifetime able to have responses to the follow-up question "Smoke Frequency"? Is this labeled in the data dictionary somewhere?

I guess I’m still confused why people that “Skipped” the parent question have responses for the “child” question. 

# Set Up

In [ ]:
dataset = 'aou-res-curation-output-prod.C2022Q4R11' #v7 w/o AIAN

# MODULES and CLIENT
from google.cloud import bigquery
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('max_colwidth', -1)
from datetime import datetime

# data

In [ ]:
def survey_data(qid, add = '', dataset = dataset):
    
    df = pd.read_gbq(f'''   
        SELECT DISTINCT person_id, concept_name as question, observation_source_concept_id question_id
        , value_source_value as answer, value_source_concept_id as answer_id, observation_datetime
        FROM `{dataset}.observation`
        JOIN `{dataset}.concept` on observation_source_concept_id = concept_id
        WHERE observation_source_concept_id = {qid}
        {add} ''')
    
    display(df.fillna('N/A').groupby(['question','question_id','answer', 'answer_id']).nunique())
    
    return df

In [ ]:
smoke_100cigs_notyes = survey_data(qid = '1585857', add = "AND value_source_value != '100CigsLifetime_Yes'")
smoke_100cigs_notyes.head()

In [ ]:
smoke_frequency = survey_data(qid = '1585860')
smoke_frequency.head()

In [ ]:
bl_issues = survey_data(qid = '1585857'
                        , add = F""" AND value_source_value != '100CigsLifetime_Yes' 
                        AND person_id IN (SELECT DISTINCT person_id
                                            FROM `{dataset}.observation`
                                            WHERE observation_source_concept_id = 1585860
                                            )""")
bl_issues.head()

In [ ]:
len(set(smoke_100cigs_notyes.person_id).intersection(set(smoke_frequency.person_id)))

In [ ]:
bl_issue_pids = set(smoke_100cigs_notyes.person_id).intersection(set(smoke_frequency.person_id))
len(bl_issue_pids)

In [ ]:
pd.DataFrame(bl_issue_pids, columns = ['person_id']).to_csv('Lifestyle_branching_logic_issues.csv', index = False)

In [ ]:
smoke_100cigs_notyes[smoke_100cigs_notyes.person_id.isin(bl_issue_pids)].sort_values()

In [ ]:
test = smoke_100cigs_notyes.merge(smoke_frequency[['person_id','observation_datetime']]
                          , on = 'person_id')#[smoke_100cigs_notyes.person_id == 1548608 ]

test['days_diff'] = test['observation_datetime_y'] - test['observation_datetime_x']
test['days_diff'] = [i.days for i in test['days_diff']]
test

In [ ]:
test[test.days_diff >0]

In [ ]:
smoke_100cigs_notyes['observation_datetime_y'] - smoke_100cigs_notyes['observation_datetime_x']

In [ ]:
smoke_frequency[smoke_frequency.person_id.isin(bl_issue_pids)]#.groupby(['question','answer']).nunique()

In [ ]:
smoke_100cigs_notyes[smoke_100cigs_notyes.person_id.isin(bl_issue_pids)].groupby(['question','answer']).nunique()

# do they have any other survey data?

In [ ]:
basics_df = pd.read_gbq(f'''   
        SELECT COUNT(DISTINCT answer) AS N_ANSWERS, question, person_id, DATE(survey_datetime) as suvey_date
        FROM `{dataset}.ds_survey`
        WHERE person_id IN {tuple(bl_issue_pids)}
        AND survey IN ('The Basics')
        GROUP BY 2,3,4
        HAVING N_ANSWERS>1
        ''')
basics_df.head()

In [ ]:
lifestyle_df = pd.read_gbq(f'''   
        SELECT COUNT(DISTINCT DATE(survey_datetime)) as n_suvey_dates, answer, question, person_id
        FROM `{dataset}.ds_survey`
        WHERE person_id IN {tuple(bl_issue_pids)}
        AND survey IN ('Lifestyle')
        GROUP BY 2,3,4
        HAVING n_suvey_dates>1
        ''')
lifestyle_df.head()